# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [4]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn', Mutes warnings when copying a slice from a DataFrame.
pd.set_option('display.max_columns', None)

In [5]:
DATA_FOLDER = '../../ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [56]:
import glob
import datetime
from io import StringIO

all_files = glob.glob(DATA_FOLDER + r'/ebola/sl_data/*.csv')

frames = []

for f in all_files:
    data = None
    
    with open(f) as file:
        data = StringIO('\n'.join([l for l in file if '%' not in l]))

    new_frame = pd.read_csv(data,
                           parse_dates=['date'], thousands=',',
                           na_values=['Nr', '-', 'NR'])
    frames.append(new_frame)
    
sl = pd.concat(frames)
newcases = ['new_suspected', 'new_probable', 'new_confirmed', 'new_positive']
deaths = ['death_suspected', 'death_probable', 'death_confirmed',
       'etc_new_deaths', 'positive_corpse']

In [57]:
sl

,34 Military Hospital,Bo,Bo EMC,Bombali,Bonthe,Hastings-F/Town,Kailahun,Kambia,Kenema,Kenema (IFRC),Kenema (KGH),Koinadugu,Kono,Moyamba,National,Police training School,Police traning School,Port Loko,Pujehun,Tonkolili,Unnamed: 18,Western area,Western area combined,Western area rural,Western area urban,date,variable
0,NaN,654142.0,NaN,494139.0,168729.0,NaN,465048.0,341690.0,653013.0,NaN,NaN,335471.0,325003.0,278119.0,6348350.0,NaN,NaN,557978.0,335574.0,434937.0,NaN,NaN,NaN,263619.0,1040888.0,2014-08-12,population
1,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,3.0,NaN,NaN,0.0,0.0,0.0,4.0,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2014-08-12,new_noncase
2,NaN,1.0,NaN,0.0,0.0,NaN,0.0,0.0,9.0,NaN,NaN,0.0,0.0,0.0,10.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2014-08-12,new_suspected
3,NaN,1.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2014-08-12,new_probable
4,NaN,0.0,NaN,0.0,0.0,NaN,0.0,0.0,9.0,NaN,NaN,0.0,0.0,0.0,11.0,NaN,NaN,2.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2014-08-12,new_confirmed
5,NaN,54.0,NaN,10.0,0.0,NaN,201.0,1.0,269.0,NaN,NaN,15.0,15.0,12.0,657.0,NaN,NaN,7.0,2.0,10.0,NaN,NaN,NaN,5.0,56.0,2014-08-12,cum_noncase
6,NaN,2.0,NaN,1.0,0.0,NaN,0.0,0.0,10.0,NaN,NaN,0.0,33.0,0.0,46.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,2014-08-12,cum_suspected
7,NaN,1.0,NaN,1.0,0.0,NaN,32.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,37.0,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,2014-08-12,cum_probable
8,NaN,22.0,NaN,7.0,1.0,NaN,378.0,1.0,259.0,NaN,NaN,0.0,1.0,5.0,717.0,NaN,NaN,24.0,3.0,2.0,NaN,NaN,NaN,1.0,13.0,2014-08-12,cum_confirmed
9,NaN,0.0,NaN,0.0,0.0,NaN,2.0,0.0,1.0,NaN,NaN,0.0,0.0,0.0,5.0,NaN,NaN,1.0,0.0,0.0,NaN,NaN,NaN,0.0,1.0,2014-08-12,death_suspected


In [60]:
sl_new = sl[(sl.variable.isin(newcases))]
di = pd.DatetimeIndex(sl_new.date);

g = sl_new.groupby([di.month, di.year])

In [73]:
g.sum()

,,34 Military Hospital,Bo,Bo EMC,Bombali,Bonthe,Hastings-F/Town,Kailahun,Kambia,Kenema,Kenema (IFRC),Kenema (KGH),Koinadugu,Kono,Moyamba,National,Police training School,Police traning School,Port Loko,Pujehun,Tonkolili,Unnamed: 18,Western area,Western area combined,Western area rural,Western area urban
date,date,,,,,,,,,,,,,,,,,,,,,,,,,
8,2014,NaN,36.0,NaN,43.0,0.0,NaN,102.0,0.0,174.0,NaN,NaN,5.0,0.0,6.0,503.0,NaN,NaN,63.0,4.0,17.0,NaN,NaN,NaN,33.0,50.0
9,2014,NaN,58.0,NaN,195.0,2.0,NaN,71.0,18.0,114.0,NaN,NaN,0.0,28.0,57.0,1180.0,NaN,NaN,240.0,14.0,72.0,NaN,NaN,NaN,152.0,221.0
10,2014,NaN,117.0,NaN,374.0,2.0,NaN,25.0,20.0,121.0,NaN,NaN,34.0,42.0,49.0,2411.0,NaN,NaN,251.0,15.0,160.0,NaN,321.0,NaN,221.0,262.0
11,2014,NaN,71.0,NaN,240.0,4.0,NaN,8.0,31.0,13.0,NaN,NaN,52.0,67.0,42.0,2367.0,NaN,NaN,261.0,6.0,105.0,NaN,NaN,NaN,248.0,502.0
12,2014,NaN,7.0,NaN,24.0,0.0,NaN,0.0,6.0,0.0,NaN,NaN,1.0,7.0,3.0,296.0,NaN,NaN,71.0,0.0,3.0,NaN,NaN,NaN,26.0,57.0


In [64]:
ndays = g['date'].nunique()
ndays

date  date
8     2014    20
9     2014    29
10    2014    28
11    2014    21
12    2014     5
Name: date, dtype: int64

In [88]:
df = g.sum().div(ndays, axis='rows')
df.dropna(axis=1, how='all')

,,Bo,Bombali,Bonthe,Kailahun,Kambia,Kenema,Koinadugu,Kono,Moyamba,National,Port Loko,Pujehun,Tonkolili,Western area,Western area rural,Western area urban
date,date,,,,,,,,,,,,,,,,
8,2014,1.800000,2.150000,0.000000,5.100000,0.000000,8.700000,0.250000,0.000000,0.300000,25.150000,3.150000,0.200000,0.850000,NaN,1.650000,2.500000
9,2014,2.000000,6.724138,0.068966,2.448276,0.620690,3.931034,0.000000,0.965517,1.965517,40.689655,8.275862,0.482759,2.482759,NaN,5.241379,7.620690
10,2014,4.178571,13.357143,0.071429,0.892857,0.714286,4.321429,1.214286,1.500000,1.750000,86.107143,8.964286,0.535714,5.714286,11.464286,7.892857,9.357143
11,2014,3.380952,11.428571,0.190476,0.380952,1.476190,0.619048,2.476190,3.190476,2.000000,112.714286,12.428571,0.285714,5.000000,NaN,11.809524,23.904762
12,2014,1.400000,4.800000,0.000000,0.000000,1.200000,0.000000,0.200000,1.400000,0.600000,59.200000,14.200000,0.000000,0.600000,NaN,5.200000,11.400000


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here